# Обучение классификатора для токсичных комментариев (мультилклассовая классификация)

Загрузка размечённого датасета

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alexandersemiletov/toxic-russian-comments")

print("Path to dataset files:", path)

100%|██████████| 12.0M/12.0M [00:01<00:00, 7.70MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/alexandersemiletov/toxic-russian-comments/versions/1


In [ ]:
%ls /root/.cache/kagglehub/datasets/alexandersemiletov/toxic-russian-comments/versions/1

dataset.txt


Загрузка библиотек для работы с трансформерами

In [ ]:
pip install torch transformers datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd

labels = ["INSULT", "OBSCENITY", "THREAT", "NORMAL"]

def parse_line(line):
    parts = line.split()
    label_part = [p.replace("__label__", "") for p in parts if p.startswith("__label__")]
    text_part = " ".join([p for p in parts if not p.startswith("__label__")])

    label_vector = {f"label_{label}": int(label in label_part) for label in labels}

    return {"text": text_part, **label_vector}

input_file = "/root/.cache/kagglehub/datasets/alexandersemiletov/toxic-russian-comments/versions/1/dataset.txt"
with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

data = [parse_line(line.strip()) for line in lines]

df = pd.DataFrame(data)

output_file = "transformed_dataset.csv"
df.to_csv(output_file, index=False, encoding="utf-8")
print(f"Датасет сохранён в {output_file}")

Датасет сохранён в transformed_dataset.csv


In [ ]:
import pandas as pd
from datasets import Dataset

dataset = Dataset.from_pandas(df)

print(dataset[0])

{'text': 'скотина! что сказать', 'label_INSULT': 1, 'label_OBSCENITY': 0, 'label_THREAT': 0, 'label_NORMAL': 0}


In [ ]:
from transformers import AutoTokenizer

model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/248290 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_dataset[0])

{'text': 'скотина! что сказать', 'label_INSULT': 1, 'label_OBSCENITY': 0, 'label_THREAT': 0, 'label_NORMAL': 0, 'input_ids': [101, 32488, 2889, 106, 1997, 21326, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
label_columns = ["label_INSULT", "label_OBSCENITY", "label_THREAT", "label_NORMAL"]

def format_labels(examples):
    return {"labels": [float(examples[col]) for col in label_columns]}

tokenized_dataset = tokenized_dataset.map(format_labels)

tokenized_dataset = tokenized_dataset.remove_columns(label_columns + ["text"])

Map:   0%|          | 0/248290 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_dataset[0])

{'input_ids': [101, 32488, 2889, 106, 1997, 21326, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=56634)

train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_columns),
    problem_type="multi_label_classification"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./rubert-toxic-classifier",
    run_name="rubert-toxic-run",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    fp16=True,
    gradient_accumulation_steps=4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2
)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = (logits > 0).astype(int)  # Применяем порог 0.5 для multi-label
    return {
        "f1": f1_score(labels, predictions, average="macro"),
        "precision": precision_score(labels, predictions, average="macro"),
        "recall": recall_score(labels, predictions, average="macro"),
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,0.214600,0.051390,0.806459,0.834325,0.787322
2,0.168000,0.045906,0.842044,0.848427,0.838853
3,0.113400,0.041873,0.858916,0.855529,0.863212
4,0.094200,0.042001,0.863751,0.864344,0.863369
5,0.078100,0.043945,0.864832,0.864896,0.864977


TrainOutput(global_step=1940, training_loss=0.18380417531913087, metrics={'train_runtime': 5055.315, 'train_samples_per_second': 196.459, 'train_steps_per_second': 0.384, 'total_flos': 6.532901704003584e+16, 'train_loss': 0.18380417531913087, 'epoch': 5.0})

In [ ]:
model.save_pretrained("rubert-toxic-classifier")
tokenizer.save_pretrained("rubert-toxic-classifier")

('rubert-toxic-classifier/tokenizer_config.json',
 'rubert-toxic-classifier/special_tokens_map.json',
 'rubert-toxic-classifier/vocab.txt',
 'rubert-toxic-classifier/added_tokens.json',
 'rubert-toxic-classifier/tokenizer.json')

In [ ]:
%rm -r rubert-toxic-classifier/checkpoint*

In [ ]:
%ls rubert-toxic-classifier/

config.json        special_tokens_map.json  tokenizer.json
model.safetensors  tokenizer_config.json    vocab.txt


In [ ]:
%%shell
tar -czvf rubert-toxic-classifier.tar.gz rubert-toxic-classifier

rubert-toxic-classifier/
rubert-toxic-classifier/model.safetensors
rubert-toxic-classifier/vocab.txt
rubert-toxic-classifier/tokenizer_config.json
rubert-toxic-classifier/special_tokens_map.json
rubert-toxic-classifier/tokenizer.json
rubert-toxic-classifier/config.json


In [ ]:
from transformers import AutoConfig

model_path = "rubert-toxic-classifier"

config = AutoConfig.from_pretrained(model_path)

id2label = {0: "INSULT", 1: "OBSCENITY", 2: "THREAT", 3: "NORMAL"}
label2id = {v: k for k, v in id2label.items()}

config.id2label = id2label
config.label2id = label2id

config.save_pretrained(model_path)

In [ ]:
from transformers import pipeline

model_path = "rubert-toxic-classifier"
classifier = pipeline(
    "text-classification",
    model=model_path,
    tokenizer=model_path,
    top_k=None
)

texts = [
    "Твою мать ебал!",
    "Сдохни в канаве",
    "Соси хуй"
]

# Предсказания для текстов
for text in texts:
    result = classifier(text)
    print(f"Text: {text}")
    print(f"Predictions: {result}")
    print("-" * 50)

Device set to use cuda:0


Text: Твою мать ебал!
Predictions: [[{'label': 'OBSCENITY', 'score': 0.4965152442455292}, {'label': 'INSULT', 'score': 0.018670054152607918}, {'label': 'THREAT', 'score': 0.014422493986785412}, {'label': 'NORMAL', 'score': 0.008913451805710793}]]
--------------------------------------------------
Text: Сдохни в канаве
Predictions: [[{'label': 'THREAT', 'score': 0.9217817783355713}, {'label': 'NORMAL', 'score': 0.019606810063123703}, {'label': 'INSULT', 'score': 0.01705537922680378}, {'label': 'OBSCENITY', 'score': 0.007157369051128626}]]
--------------------------------------------------
Text: Соси хуй
Predictions: [[{'label': 'OBSCENITY', 'score': 0.6891267895698547}, {'label': 'INSULT', 'score': 0.06637974828481674}, {'label': 'THREAT', 'score': 0.009389976970851421}, {'label': 'NORMAL', 'score': 0.00875417236238718}]]
--------------------------------------------------
